In [ ]:
"Apr.26"

In [59]:
import scipy.io as scipy
import os
import os.path as osp
import pandas as pd
import numpy as np
import pickle
import networkx as nx
from tqdm import tqdm

In [32]:
BASEDIR = os.getcwd()
MAP_NAME = 'NYC_Manhattan_Map.mat'
REQ_NAME = 'Requests.mat'
OUTPUT_NAME = 'Manhattan_Map' 

In [33]:
map = scipy.loadmat(osp.join(BASEDIR,MAP_NAME))

In [34]:
map_Arcs = pd.DataFrame(map['Arcs']) #ArcsID, Node_Origin, Node_destination, starts from 1 
map_CityArcs = pd.DataFrame(map['CityArcs']) #NodeID, NodeID, starting from 0, 1 means connected.
map_EdgeTimes = pd.DataFrame(map['EdgeTimes']) #ArcTime in seconds, position is ArcID
map_Nodes = pd.DataFrame(map['Nodes']) # Node ID, latitude, longtitude

In [70]:
map_Arcs.rename(columns={0:'ArcID',1:'Oid',2:'Did'},inplace=True)
map_EdgeTimes.rename(columns={0:'ArcTime'},inplace=True)
map_Nodes.rename(columns={0:'NodeID',1:'Latitude',2:'Longitude'},inplace=True)

In [71]:
# Add self-Connection to map_CityArcs
for i in range(map_Nodes.shape[0]):
    map_CityArcs.loc[i,i] = 1

# Change NodeID to Int
map_Nodes['NodeID'] = map_Nodes['NodeID'].astype(int)

In [78]:
map_Arcs.iloc[0]

ArcID    1
Oid      1
Did      4
Name: 0, dtype: uint16

In [88]:
# Generate Graph
G = nx.DiGraph()
num_edges = len(map_Arcs) #num of edges in the map
arcs = tqdm(map_Arcs.iterrows(), total=num_edges, ncols=100, desc='Building network...') #ncols is the width of the progress bar

for idx, arcs in arcs:
    Oid = arcs['Oid']
    Did = arcs['Did']
    ArcID = arcs['ArcID']-1 #EdgeTime is indexed from 0
    G.add_node(Oid) #add current node to the graph
    G.add_edge(Oid, Did, TimeCost=map_EdgeTimes.iloc[ArcID]['ArcTime']) #add the arc to the graph

print('Network data loaded.')
print(f'Network has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.')

Building network...: 100%|███████████████████████████████████| 9452/9452 [00:00<00:00, 15769.89it/s]

Network data loaded.
Network has 4091 nodes and 9452 edges.


In [ ]:
test

In [ ]:
# Compile to All_Path_table
# All_Path_table[Oid][Did][0]: Path
# All_Path_table[Oid][Did][1]: Time

In [56]:
# Load Requests File
req = scipy.loadmat(osp.join(BASEDIR,REQ_NAME))
req_matrix = pd.DataFrame(req['Requests']) #Orid, Did, ReqTime(mins, need to add random secs for this), size of req

In [57]:
req_matrix.rename(columns={0:'Oid',1:'Did',2:'ReqTime',3:'Size'},inplace=True)
req_matrix

,Oid,Did,ReqTime,Size
0,1023,1569,0,6
1,2429,2218,0,1
2,1627,3942,0,5
3,2561,2706,0,1
4,2364,2834,0,2
...,...,...,...,...
10769,1107,2597,59,5
10770,1556,483,59,1
10771,2301,628,59,3
10772,805,1181,59,1


In [58]:
# Convert ReqTime to seconds
req_matrix['ReqTime'] = req_matrix['ReqTime'] * 60

# Generate random numbers between 0 and 59
random_seconds = np.random.randint(0, 60, size=len(req_matrix))

# Add random seconds to ReqTime column
req_matrix['ReqTime'] += random_seconds

req_matrix


,Oid,Did,ReqTime,Size
0,1023,1569,8,6
1,2429,2218,31,1
2,1627,3942,36,5
3,2561,2706,12,1
4,2364,2834,27,2
...,...,...,...,...
10769,1107,2597,3588,5
10770,1556,483,3570,1
10771,2301,628,3568,3
10772,805,1181,3595,1
